In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)


(True, 'Tesla T4')

In [ ]:
# Cell 2: mount Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Cell 3: clone YOLOv5 and install its dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17413, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17413 (delta 64), reused 22 (delta 22), pack-reused 17327 (from 4)
Receiving objects: 100% (17413/17413), 16.30 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (11935/11935), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s e

In [ ]:
# Cell 4: sample 400 images + labels into a local dataset folder
import os, random, shutil
from pathlib import Path

# original data
orig_img_dir = '/content/drive/My Drive/AI_5/Training_Set/train'
orig_lbl_dir = '/content/drive/My Drive/AI_5/Training_Set/labels_yolo'

# where YOLO expects them
DATA_ROOT    = Path('/content/dataset')
train_img_dir = DATA_ROOT/'images'/'train'
train_lbl_dir = DATA_ROOT/'labels'/'train'
train_img_dir.mkdir(parents=True, exist_ok=True)
train_lbl_dir.mkdir(parents=True, exist_ok=True)

# pick 400 random images
all_imgs = [f for f in os.listdir(orig_img_dir)
            if f.lower().endswith(('.jpg','.jpeg','.png'))]
train_imgs = random.sample(all_imgs, 400)

# copy images and their .txt labels
for img in train_imgs:
    shutil.copy(Path(orig_img_dir)/img,   train_img_dir/img)
    lbl = img.rsplit('.',1)[0] + '.txt'
    shutil.copy(Path(orig_lbl_dir)/lbl,   train_lbl_dir/lbl)

print(f"Sampled {len(train_imgs)} images to {train_img_dir}")


Sampled 400 images to /content/dataset/images/train


In [ ]:
# Cell 4: write data.yaml for YOLO
import yaml

# class “graffiti”:
classes = ['graffiti']

data_yaml = {
    'train': str(train_img_dir),
    'val':   str(train_img_dir),  # re-using train as val for now
    'nc':    len(classes),
    'names': classes
}
with open('data.yaml','w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print("data.yaml created:")
print(open('data.yaml').read())


data.yaml created:
train: /content/dataset/images/train
val: /content/dataset/images/train
nc: 1
names:
- graffiti



In [ ]:
#Cell 5

!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 50 \
  --data data.yaml \
  --weights yolov5s.pt \
  --project graffiti_detection \
  --name exp1


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-30 16:01:06.228226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746028866.466132    5174 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746028866.526857    5174 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
#Cell 6
!ls /content/yolov5/graffiti_detection/exp1/weights


best.pt  last.pt


In [ ]:
#Cell 7

from pathlib import Path
import shutil

src = Path('/content/yolov5/graffiti_detection/exp1/weights/best.pt')
dst = Path('/content/drive/My Drive/AI_5/Training_Set/best.pt')

dst.parent.mkdir(parents=True, exist_ok=True)
shutil.copy(src, dst)

print(f"Copied best.pt to {dst}")


Copied best.pt to /content/drive/My Drive/AI_5/Training_Set/best.pt


In [ ]:
# Cell 8: convert single test‐labels CSV into per‐image YOLO .txt files
import pandas as pd
import os
from pathlib import Path

# paths
test_csv = Path('/content/drive/My Drive/AI_5/Testing_Set/labels/test_labels.csv')
img_dir  = Path('/content/drive/My Drive/AI_5/Testing_Set/test')
out_lbl  = Path('/content/drive/My Drive/AI_5/Testing_Set/labels_yolo')

out_lbl.mkdir(parents=True, exist_ok=True)

# load and assume columns: filename,width,height,class,xmin,ymin,xmax,ymax
df = pd.read_csv(test_csv)
classes = sorted(df['class'].unique())
cls_map = {c:i for i,c in enumerate(classes)}

for fname, g in df.groupby('filename'):
    w_img, h_img = g.iloc[0][['width','height']]
    lines = []
    for _, row in g.iterrows():
        xmin,ymin,xmax,ymax = row[['xmin','ymin','xmax','ymax']]
        x_c = ((xmin + xmax)/2)/w_img
        y_c = ((ymin + ymax)/2)/h_img
        bw  = (xmax - xmin)/w_img
        bh  = (ymax - ymin)/h_img
        cls_id = cls_map[row['class']]
        lines.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")
    with open(out_lbl/f"{Path(fname).stem}.txt","w") as f:
        f.write("\n".join(lines))


In [ ]:
# Cell 9: helper functions for IoU and box conversions
import cv2
import numpy as np

def xywhn_to_xyxy(x, y, w, h, W, H):
    """YOLO normalized → absolute xyxy."""
    xc, yc = x*W, y*H
    bw, bh = w*W, h*H
    x1 = xc - bw/2
    y1 = yc - bh/2
    x2 = xc + bw/2
    y2 = yc + bh/2
    return [x1, y1, x2, y2]

def compute_iou(boxA, boxB):
    """Compute IoU of two boxes [x1,y1,x2,y2]."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = (boxA[2]-boxA[0])*(boxA[3]-boxA[1])
    boxBArea = (boxB[2]-boxB[0])*(boxB[3]-boxB[1])
    return interArea / (boxAArea + boxBArea - interArea + 1e-6)


In [ ]:
# Cell 10: one‐pass inference on 40 random test images and save results.csv + two best examples


import cv2
import torch
import random
import shutil
import csv
from pathlib import Path


# paths & parameters
DRIVE      = Path('/content/drive/My Drive/AI_5')
TEST_IMG   = DRIVE/'Testing_Set'/'test'
TEST_LBL   = DRIVE/'Testing_Set'/'labels_yolo'
BEST_MODEL = DRIVE/'Training_Set'/'best.pt'
OUT_ITER   = DRIVE/'Testing_Set'/'iteration'/'iteration_1'

# prepare output folder
shutil.rmtree(OUT_ITER, ignore_errors=True)
(OUT_ITER/'examples').mkdir(parents=True, exist_ok=True)

# sample 40 test images
all_test    = sorted([f.name for f in TEST_IMG.iterdir() if f.suffix.lower() in ('.jpg','.png','.jpeg')])
test_sample = random.sample(all_test, 40)

# load the custom-trained YOLOv5 model
model = torch.hub.load('/content/yolov5', 'custom', path=str(BEST_MODEL), source='local')
model.conf = 0.0  # do not filter by confidence

results = []

for fname in test_sample:
    img_path = TEST_IMG/fname
    # load image to get dimensions
    img = cv2.imread(str(img_path))
    W, H = img.shape[1], img.shape[0]

    # read only the first annotation line
    with open(TEST_LBL/f"{Path(fname).stem}.txt") as f:
        parts = f.readline().strip().split()
    # parts = [cls_id, x_center, y_center, width, height]
    _, x, y, w, h = map(float, parts)
    gt_box = xywhn_to_xyxy(x, y, w, h, W, H)

    # run inference
    preds = model(str(img_path)).xyxy[0]  # tensor Nx6: x1,y1,x2,y2,conf,cls
    if len(preds) == 0:
        conf, iou = 0.0, 0.0
    else:
        top = preds[preds[:, 4].argmax()]
        pb   = top[:4].tolist()
        conf = float(top[4])
        iou  = compute_iou(pb, gt_box)

    results.append((fname, conf, iou))

# write results.csv
with open(OUT_ITER/'results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_name', 'confidence', 'iou'])
    writer.writerows(results)

# save top-2 IoU examples (ground-truth in green, prediction in red)
for fname, conf, iou in sorted(results, key=lambda x: x[2], reverse=True)[:2]:
    img = cv2.imread(str(TEST_IMG/fname))
    # draw ground-truth box
    with open(TEST_LBL/f"{Path(fname).stem}.txt") as f:
        parts = f.readline().strip().split()
    _, x, y, w, h = map(float, parts)
    gt = xywhn_to_xyxy(x, y, w, h, W, H)
    x1, y1, x2, y2 = map(int, gt)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # draw prediction box if present
    preds = model(str(TEST_IMG/fname)).xyxy[0]
    if len(preds) > 0:
        pred  = preds[preds[:, 4].argmax()][:4].tolist()
        x1p, y1p, x2p, y2p = map(int, pred)
        cv2.rectangle(img, (x1p, y1p), (x2p, y2p), (0, 0, 255), 2)

    cv2.imwrite(str(OUT_ITER/'examples'/fname), img)



YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.p

In [ ]:

# Cell 11: full iterative training + testing loop with dynamic run-dir detection
import random
import shutil
import csv
import yaml
import cv2
import torch
from pathlib import Path

# helper functions
def xywhn_to_xyxy(x, y, w, h, W, H):
    xc, yc = x*W, y*H
    bw, bh = w*W, h*H
    return [xc - bw/2, yc - bh/2, xc + bw/2, yc + bh/2]

def compute_iou(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    interW, interH = max(0, xB - xA), max(0, yB - yA)
    interArea = interW * interH
    boxAArea = (boxA[2]-boxA[0]) * (boxA[3]-boxA[1])
    boxBArea = (boxB[2]-boxB[0]) * (boxB[3]-boxB[1])
    return interArea / (boxAArea + boxBArea - interArea + 1e-6)

# paths
DRIVE      = Path('/content/drive/My Drive/AI_5')
TRAIN_IMG  = DRIVE/'Training_Set'/'train'
TRAIN_LBL  = DRIVE/'Training_Set'/'labels_yolo'
TEST_IMG   = DRIVE/'Testing_Set'/'test'
TEST_LBL   = DRIVE/'Testing_Set'/'labels_yolo'
BEST_BASE  = DRIVE/'Training_Set'/'best.pt'
ITER_ROOT  = DRIVE/'Testing_Set'/'iteration'

classes = ['graffiti']
nc = len(classes)

all_train = set(f.name for f in TRAIN_IMG.iterdir() if f.suffix.lower() in ('.jpg','.png','.jpeg'))
all_test  = set(f.name for f in TEST_IMG.iterdir()  if f.suffix.lower() in ('.jpg','.png','.jpeg'))
used_train = set()
used_test  = set()
prev_best = BEST_BASE

iteration = 1
while True:
    if len(all_train - used_train) < 400 or len(all_test - used_test) < 40:
        print("Not enough images remaining; stopping.")
        break

    # sample fresh splits
    train_split = random.sample(list(all_train - used_train), 400)
    test_split  = random.sample(list(all_test - used_test),   40)
    used_train.update(train_split)
    used_test .update(test_split)

    # prepare iteration folder
    this_iter = ITER_ROOT/f"iteration_{iteration}"
    shutil.rmtree(this_iter, ignore_errors=True)
    (this_iter/'train').mkdir(parents=True)
    (this_iter/'test').mkdir(parents=True)
    (this_iter/'examples').mkdir(parents=True)

    # copy training data
    for fname in train_split:
        shutil.copy(TRAIN_IMG/fname, this_iter/'train'/fname)
        shutil.copy(TRAIN_LBL/f"{Path(fname).stem}.txt",
                    this_iter/'train'/f"{Path(fname).stem}.txt")

    # copy test data
    for fname in test_split:
        shutil.copy(TEST_IMG/fname, this_iter/'test'/fname)
        shutil.copy(TEST_LBL/f"{Path(fname).stem}.txt",
                    this_iter/'test'/f"{Path(fname).stem}.txt")

    # write data.yaml
    data_yaml = {
        'train': str(this_iter/'train'),
        'val':   str(this_iter/'test'),
        'nc':    nc,
        'names': classes
    }
    with open('data.yaml','w') as f:
        yaml.dump(data_yaml, f, sort_keys=False)

    # train (YOLOv5 will create a subfolder under this_iter)
    get_ipython().system(f"""
      python /content/yolov5/train.py \
        --img 640 \
        --batch 16 \
        --epochs 50 \
        --data data.yaml \
        --weights "{prev_best}" \
        --project "{this_iter}" \
        --name "train"
    """)

    # locate the actual run directory (train, train2, etc.)
    run_dirs = [d for d in this_iter.iterdir() if (d/'weights'/'best.pt').exists()]
    if not run_dirs:
        raise FileNotFoundError(f"No best.pt found in any subdir of {this_iter}")
    run_dir = run_dirs[0]
    prev_best = run_dir/'weights'/'best.pt'

    # load model for inference
    model = torch.hub.load('/content/yolov5', 'custom',
                           path=str(prev_best), source='local')
    model.conf = 0.0

    # inference + IoU + CSV + examples
    results = []
    for fname in test_split:
        img_path = this_iter/'test'/fname
        img = cv2.imread(str(img_path))
        H, W = img.shape[:2]

        # read GT
        with open(this_iter/'test'/f"{Path(fname).stem}.txt") as f:
            parts = f.readline().split()
        _, x, y, w, h = map(float, parts)
        gt_box = xywhn_to_xyxy(x, y, w, h, W, H)

        preds = model(str(img_path)).xyxy[0]
        if len(preds) == 0:
            conf, iou = 0.0, 0.0
        else:
            top = preds[preds[:,4].argmax()]
            conf = float(top[4])
            pb   = top[:4].tolist()
            iou  = compute_iou(pb, gt_box)

        results.append((fname, conf, iou))

    # save results.csv
    with open(this_iter/'results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['image_name','confidence','iou'])
        writer.writerows(results)

    # save top-2 IoU examples
    for fname, _, _ in sorted(results, key=lambda x: x[2], reverse=True)[:2]:
        img = cv2.imread(str(this_iter/'test'/fname))

        # draw GT
        with open(this_iter/'test'/f"{Path(fname).stem}.txt") as f:
            parts = f.readline().split()
        _, x, y, w, h = map(float, parts)
        gt = xywhn_to_xyxy(x, y, w, h, W, H)
        x1, y1, x2, y2 = map(int, gt)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)

        # draw pred
        preds = model(str(this_iter/'test'/fname)).xyxy[0]
        if len(preds) > 0:
            p = preds[preds[:,4].argmax()][:4].tolist()
            x1p, y1p, x2p, y2p = map(int, p)
            cv2.rectangle(img, (x1p,y1p), (x2p,y2p), (0,0,255), 2)

        cv2.imwrite(str(this_iter/'examples'/fname), img)

    # stopping criterion
    high = sum(1 for _, _, iou in results if iou >= 0.90)
    pct  = high / len(results)
    print(f"Iteration {iteration}: {high}/40 images IoU≥0.90 → {pct*100:.1f}%")
    if pct >= 0.80:
        print("Stopping: criterion met.")
        break

    iteration += 1




wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-30 17:46:21.052587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746035181.073339   32358 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746035181.079722   32358 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=/content/drive/My Drive/AI_5/Training_Set/best.pt, cfg=, data=d

YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.p

Iteration 1: 9/40 images IoU≥0.90 → 22.5%
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-30 18:03:44.993529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746036225.026662   36909 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746036225.036758   36909 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=/content/drive/My Dri

YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.p

Iteration 2: 18/40 images IoU≥0.90 → 45.0%
Not enough images remaining; stopping.
